In [5]:
import cv2
import pandas as pd
import numpy as np
import sys

sys.path.append('../')
from functions.video_processing import get_coordinates_from_video

# set random seed for reproducibility
np.random.seed(42)

# screen resolution
monitor_width = 3440
monitor_height = 1440

I0000 00:00:1736529310.308237  827699 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-20.2.48), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1736529310.652696  836105 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736529310.708012  836105 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [2]:
def rescale_frame(frame, scale):

    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)

    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)


def find_scale_factor(frame, monitor_width, monitor_height):
    """
    Rescales the frame to the size of monitor
    """

    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

    if (frame_width > monitor_width) or (frame_height > monitor_height):
        with_scale = monitor_width / frame_width
        height_scale = monitor_height / frame_height
        return min(with_scale, height_scale)
    
    if (frame_width < monitor_width) and (frame_height < monitor_height):
        with_scale = monitor_width / frame_width
        height_scale = monitor_height / frame_height
        return min(with_scale, height_scale)

    return 1
    

In [6]:
df_frames = pd.read_csv('../data/tmp_tracking.csv')

### Pedal position for knee-over-pedal calculations

In [7]:
pedal_coordinates_sampled = []
tube_coordinates_sampled = []
frames_parallel_pedals = df_frames[df_frames['parallel_pedals'] == True]

# sample a frame from frames_parallel_pedals
sample_frames = frames_parallel_pedals.sample(3)['frame'].values

In [8]:
pedal_coordinates_sampled = get_coordinates_from_video(sample_frames, monitor_width, monitor_height, msg='', type_coordinates='point')


iteration 1 of 3 - Captured coordinates (362, 1046), press any key to continue
iteration 2 of 3 - Captured coordinates (362, 1035), press any key to continue
iteration 3 of 3 - Captured coordinates (347, 1037), press any key to continue
Pedal coordinates obtained (original frame sizes): [158 461]


In [5]:
for i, sample_frame in enumerate(sample_frames):
    img = cv2.imread(f'../data/tmp_frames/frame_{sample_frame}.jpg')

    # does not depend on the monitor resolution
    scale = find_scale_factor(img, monitor_width, monitor_height)
    img_rescaled = rescale_frame(img, scale=scale)


    def capture_event(event, x, y, flags, param):
        captured = False

        # make a copy of the image
        img_rescaled_tmp = img_rescaled.copy()

        if event == cv2.EVENT_MOUSEMOVE:

            # plot a green vertical and horizontal line
            cv2.line(img_rescaled_tmp, (x, 0), (x, img_rescaled.shape[0]), (0, 255, 0), 1)
            cv2.line(img_rescaled_tmp, (0, y), (img_rescaled.shape[1], y), (0, 255, 0), 1)

            cv2.imshow("frame", img_rescaled_tmp)

        if event == cv2.EVENT_LBUTTONDOWN:
            print(f"iteration {i + 1} of 3 - Captured coordinates ({x}, {y}), press any key to continue")
            pedal_coordinates_sampled.append((x, y))
            cv2.destroyWindow("frame")


    cv2.imshow('frame', img_rescaled)
    cv2.setMouseCallback('frame', capture_event)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# pedal coordinates obtained in the resized monitor
pedal_coordinates = ((1 / scale) * np.array(pedal_coordinates_sampled).mean(axis=0)).astype(int)

print(f"Pedal coordinates obtained (original frame sizes): {pedal_coordinates}")

iteration 1 of 3 - Captured coordinates (366, 1040), press any key to continue
iteration 2 of 3 - Captured coordinates (362, 1039), press any key to continue
iteration 3 of 3 - Captured coordinates (367, 1035), press any key to continue
Pedal coordinates obtained (original frame sizes): [162 461]


### Tube length identification

In [6]:
img = cv2.imread(f'../data/tmp_frames/frame_{sample_frames[0]}.jpg')

# does not depend on the monitor resolution
img_rescaled = rescale_frame(img, scale=scale)


def capture_event(event, x, y, flags, param):
    captured = False

    # make a copy of the image
    img_rescaled_tmp = img_rescaled.copy()

    if event == cv2.EVENT_MOUSEMOVE:

        # plot a green vertical and horizontal line
        cv2.line(img_rescaled_tmp, (x, 0), (x, img_rescaled.shape[0]), (0, 255, 0), 1)
        cv2.line(img_rescaled_tmp, (0, y), (img_rescaled.shape[1], y), (0, 255, 0), 1)
        cv2.imshow("frame", img_rescaled_tmp)

    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Captured tube coordinates ({x}, {y}), press any key to continue")
        tube_coordinates_sampled.append((x, y))


        if len(tube_coordinates_sampled) == 2:
            cv2.line(img_rescaled_tmp, tube_coordinates_sampled[0], tube_coordinates_sampled[1], (0, 0, 255), 2)
            cv2.imshow("frame", img_rescaled_tmp)
            cv2.waitKey(200)
            cv2.destroyWindow("frame")

            print(f'Tube coordinates obtained (rescaled image): {tube_coordinates_sampled}')
            print('press any key to continue ...')


cv2.imshow('frame', img_rescaled)
cv2.setMouseCallback('frame', capture_event)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

tube_coordinates_sampled = ((1 / scale) * np.array(tube_coordinates_sampled)).astype(int)

Captured tube coordinates (297, 808), press any key to continue
Captured tube coordinates (488, 854), press any key to continue
Tube coordinates obtained (rescaled image): [(297, 808), (488, 854)]
press any key to continue ...


In [7]:
tube_pixel_size = np.linalg.norm(tube_coordinates_sampled[0] - tube_coordinates_sampled[1])

In [8]:
tube_pixel_size

86.34813257969161